## The CBS Secret Sauce Investigation: Scraping & Cleaning

#### Creator: Mitch Brinkman

In [1]:
from bs4 import BeautifulSoup
import requests

In [2]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf
import patsy
import re
import pickle
import string
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import word_tokenize, TreebankWordTokenizer
from nltk.stem import PorterStemmer, WordNetLemmatizer 
from nltk.util import ngrams
from nltk.corpus import stopwords
import nltk
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /Users/user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/user/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [30]:
from sklearn.feature_extraction import text 
from sklearn.feature_extraction.text import CountVectorizer

In [31]:
import sys
sys.setrecursionlimit(20000)

In [60]:
from cbs_func import clean_sentiment

ImportError: cannot import name 'clean_sentiment' from 'cbs_func' (/Users/user/Desktop/notes_chi20_ds14/CBS_TV_project4/cbs_tv_nlp_proj4/notebooks/cbs_func.py)

In [55]:
from cbs_func import make_ep_list
from cbs_func import clean_tv_scripts
from cbs_func import ncis_data
from cbs_func import shield_data


ImportError: cannot import name 'clean_sentiment' from 'cbs_func' (/Users/user/Desktop/notes_chi20_ds14/CBS_TV_project4/cbs_tv_nlp_proj4/notebooks/cbs_func.py)

### Web Scraping

In [38]:
url_list = ['https://subslikescript.com/series/NCIS-364845','https://subslikescript.com/series/The_Shield-286486']

#### NCIS scripts

In [35]:
ncis_list = make_ep_list(url_list[0])

In [36]:
for i, num in enumerate(ncis_list):
    ncis_list[i] = num.replace('/series/NCIS-364845/','')

In [37]:
ncis_df = ncis_data(ncis_list)

In [47]:
with open('./data/raw/ncis_series.pkl', 'wb') as f:
    pickle.dump(ncis_df, f)

#### The Shield scripts

In [39]:
shield_list = make_ep_list(url_list[1])

In [40]:
for i, num in enumerate(shield_list):
    shield_list[i] = num.replace('/series/The_Shield-286486/','')

In [41]:
shield_df = shield_data(shield_list)

In [61]:
with open('./data/raw/shield_series.pkl', 'wb') as f:
    pickle.dump(shield_df, f)

FileNotFoundError: [Errno 2] No such file or directory: './data/raw/shield_series.pkl'

### Cleaning

In [79]:
add_stop_words = ['tony','abby','dinozzo','uh','um']
stop_words = text.ENGLISH_STOP_WORDS.union(add_stop_words)

In [3]:
def clean_tv_scripts(text):
    '''Make text lowercase, remove text in square brackets, 
    remove punctuation and remove words containing numbers.'''
    text = re.sub('([A-Z]{2,})', ' ', text)
    text = text.lower()
    text = re.sub('\[.*?\]', ' ', text)
    text = re.sub('\<.*?>', ' ', text)
    text = re.sub('\(.*?\)', ' ', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub('\n', ' ', text)
    text = re.sub('♪', '', text)
    text = re.sub('  ', ' ', text)
    text = re.sub('   ', ' ', text)
    text = re.sub('    ', ' ', text)
    return text

big_wash = lambda x: clean_tv_scripts(x)

#### Corpus Prep for Sentiment Analysis

In [92]:
def clean_sentiment(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation 
    and remove words containing numbers.'''
    text = re.sub('([A-Z]{2,})', ' ', text)
    text = re.sub('\[.*?\]', ' ', text)
    text = re.sub('\<.*?>', ' ', text)
    text = re.sub('\(.*?\)', ' ', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub('\n', ' ', text)
    text = re.sub('♪', '', text)
    return text

first_wash = lambda x: clean_sentiment(x)

In [93]:
# # Pickle reload of raw file, for easy drop into notebook

# ncis_df = pd.read_pickle('/Users/user/Desktop/notes_chi20_ds14/CBS_TV_project4/cbs_tv_nlp_proj4/data/raw/ncis_series.pkl')
# shield_df = pd.read_pickle('/Users/user/Desktop/notes_chi20_ds14/CBS_TV_project4/cbs_tv_nlp_proj4/data/raw/shield_series.pkl')

In [94]:
clean_ncis = pd.DataFrame(ncis_df.dialogue.apply(first_wash))
clean_shield = pd.DataFrame(shield_df.dialogue.apply(first_wash))

In [96]:
with open('/Users/user/Desktop/notes_chi20_ds14/CBS_TV_project4/cbs_tv_nlp_proj4/data/raw/shield_series.pkl', 'wb') as f:
    pickle.dump(shield_df, f)
clean_ncis.to_pickle('ncis_sentiment.pkl')
clean_shield.to_pickle('shield_sentiment.pkl')

In [11]:
ncis_df = pd.read_pickle('./pickles/ncis_full_series.pkl')
bosch_df = pd.read_pickle('./pickles/bosch_series.pkl')
shield_df = pd.read_pickle('./pickles/shield_series.pkl')

In [73]:
ncis_df.set_index('ep_id',inplace=True)
bosch_df.set_index('ep_id',inplace=True)
shield_df.set_index('ep_id',inplace=True)

In [74]:
ncis_df = ncis_df.astype(str)
bosch_df = bosch_df.astype(str)
shield_df = shield_df.astype(str)

In [75]:
ncis_df.to_pickle('ncis_full_series.pkl')
bosch_df.to_pickle('bosch_series.pkl')
shield_df.to_pickle('shield_series.pkl')

In [76]:
ncis_df.head()

,dialogue
ep_id,
ncis_season-1/episode-1-Yankee_White,"<div class=""full-script""><br/>(ENGINES WHIRRIN..."
ncis_season-1/episode-2-Hung_Out_to_Dry,"<div class=""full-script""><br/>(GHOST OF A GOOD..."
ncis_season-1/episode-3-Seadog,"<div class=""full-script""><br/>(WHAT I LIKE ABO..."
ncis_season-1/episode-4-The_Immortals,"<div class=""full-script""><br/>(REGGAE MUSIC PL..."
ncis_season-1/episode-5-The_Curse,"<div class=""full-script""><br/>Shoot.<br/><br/>..."


In [89]:
# Let's take a look at the updated text
clean_ncis = pd.DataFrame(ncis_df.dialogue.apply(first_wash))
clean_bosch = pd.DataFrame(bosch_df.dialogue.apply(first_wash))
clean_shield = pd.DataFrame(shield_df.dialogue.apply(first_wash))

In [80]:
clean_ncis.head(10)

,dialogue
ep_id,
ncis_season-1/episode-1-Yankee_White,bobby those baby backs from papa joes ...
ncis_season-1/episode-2-Hung_Out_to_Dry,sarah jimmy jimmy stop i said no god ...
ncis_season-1/episode-3-Seadog,yo did you hear that yeah ive always ...
ncis_season-1/episode-4-The_Immortals,yeah take it easy come on dude did...
ncis_season-1/episode-5-The_Curse,shoot grab your gear my three favourit...
ncis_season-1/episode-6-High_Seas,this place is on fire nice accent...
ncis_season-1/episode-7-Sub_Rosa,i cannot believe they lost i cant belie...
ncis_season-1/episode-8-Minimum_Security,woman you promised id be back by now dont g...
ncis_season-1/episode-9-Marine_Down,laura they wouldnt even tell sarah how he...


In [90]:
clean_ncis.dialogue.loc['ncis_season-11/episode-3-Under_the_Radar']

'  ripped by mstoll looking for terry yeah registered letter for him mailman brought it late yesterday afternoon i signed for it give it to me ill put it in his apartment thanks i dont wanna be a pain but ive still got ants in my kitchen exterminators coming this afternoon theyll be done before you get back from class i promise  thanks thats what im here for  hey murray what are you doing at zivas desk officially not her desk anymore agent dinozzo is gibbs going to replace her i dont know its only been a week he hasnt really discussed it with me  i repeat murray what are you doing just scrubbing her hard drive  this kind of looks like you it is  how close were you two give me that morning agent mcgee hey tim whats the matter cant find my badge and  ooh lost your creds thats serious yeah i know tony i couldnt find them at home  i was hoping id left them here oh youre so screwed thank god i didnt lose my buildingaccess card lost creds initiate an automatic  investigation almost as seriou

In [91]:
clean_ncis.to_pickle('clean_ncis_full_series.pkl')
clean_bosch.to_pickle('clean_bosch_series.pkl')
clean_shield.to_pickle('clean_shield_series.pkl')

In [108]:
clean_ncis = clean_ncis.astype(str)

In [127]:
clean_bosch.tail()

,dialogue
ep_id,
bosch_season-3/episode-9-Clear_Shot,this is air were over here at the help call b...
bosch_season-3/episode-10-The_Sea_King,who eats a chili fucking dog before going out...
bosch_season-5/episode-1-Episode_51,last season on bosch it is essential we clear...
bosch_season-5/episode-2-Episode_52,wipe it down thanks motherfucker think hell d...
bosch_season-5/episode-4-Episode_54,i got a feeling that i cant let go i got a...


In [128]:
clean_bosch.dialogue.loc['bosch_season-5/episode-4-Episode_54']

'   i got a feeling that i cant let go  i got a feeling that i cant let go  i got a feeling that i cant let go  cant let go  i got a feeling that i cant let go  cant let go cant let go  i got a feeling  that i cant let go  i got a feeling that i cant let go  cant let go cant let go  i got a feeling  that i cant let go  cant let go  cant let go  like me found your calling dont even have to glove up little early for bustin balls just try not to hurt anybody with that thing powers  fucking bosch heard that scrote youre up bright and early took a hard look at that socalled confession its bogus not his mo olmer was a serial rapist not a murderer well he could have graduated to it natural progression olmer claims he climbed skylers balcony slipped through an unlocked sliding door how he attacked his rape victims the railing was covered with a layer of soot so was the deck no handprints no footprints no one came in from the balcony good what else no mention of the carving knife carving knife 

In [ ]:
# shows = [ncis_df,bosch_df,shield_df]
# tv_df = pd.concat(shows).reset_index(drop=True)
# tv_df = tv_df.astype(str)

In [ ]:
# tv_dict = dict(zip(tv_df.ep_id, tv_df.dialogue))

In [ ]:
# type(tv_dict['bosch_season-2/episode-5-Gone'])

In [ ]:
show_name = ['ncis','ncis','ncis','ncis','ncis','bosch','bosch','bosch','bosch','bosch','the shield','the shield','the shield','the shield','the shield',]
tv_df['show_name'] = show_name


In [ ]:
clean_tv_names = pd.read_pickle('clean_tv_names.pkl')

In [ ]:
clean_tv_names

In [ ]:
# Let's take a look at the updated text
clean_tv_names = pd.DataFrame(tv_df.dialogue.apply(first_wash))
clean_tv_names

In [ ]:
clean_tv_names.dialogue.loc['ncis_season-11/episode-3-Under_the_Radar']

In [ ]:
print(test)

In [ ]:
cv = CountVectorizer(stop_words='english')
clean_tv_cv = cv.fit_transform(clean_tv.dialogue)
tv_dtm = pd.DataFrame(clean_tv_cv.toarray(), columns=cv.get_feature_names())
tv_dtm.index = clean_tv.index
tv_dtm

In [ ]:
tv_dtm.to_pickle('tv_dtm.pkl')

In [ ]:
clean_tv.to_pickle('clean_tv.pkl')
pickle.dump(cv, open("cv.pkl", "wb"))

In [153]:
# class NLPProcessor:
    
#     def __init__(self, vectorizer_class, tokenizer_function, cleaning_function, stemmer_function):
#         self.vectorizer = vectorizer_class
#         self.tokenizer = tokenizer_function
#         self.cleaning_function = cleaning_function
#         self.stemmer = stemmer_function
    
#     def fit(self, corpus_list_to_fit):
#         cleaned_corpus = list(map(self.cleaning_function, corpus_list_to_fit))
#         print(cleaned_corpus)
#         tokenized_list = list(map(self.tokenizer, cleaned_corpus))
#         print(tokenized_list)
#         stemmed_list = [' '.join(list(map(self.stemmer, item))) for item in tokenized_list]
#         print(stemmed_list)
#         return self.vectorizer.fit(stemmed_list)
    
#     def transform(self, corpus_list_to_clean):
#         cleaned_corpus = list(map(self.cleaning_function, corpus_list_to_clean))
#         tokenized_list = list(map(self.tokenizer, cleaned_corpus))
#         stemmed_list = [' '.join(list(map(self.stemmer, item))) for item in tokenized_list]
#         return pd.DataFrame(self.vectorizer.transform(stemmed_list).toarray(), 
#                             columns=self.vectorizer.get_feature_names())

#### NLP Pipeline Processing

In [164]:
ncis_df = pd.read_pickle('ncis_full_series.pkl')
bosch_df = pd.read_pickle('bosch_series.pkl')
shield_df = pd.read_pickle('shield_series.pkl')

In [165]:
ncis_df.head()

,dialogue
ep_id,
ncis_season-1/episode-1-Yankee_White,"<div class=""full-script""><br/>(ENGINES WHIRRIN..."
ncis_season-1/episode-2-Hung_Out_to_Dry,"<div class=""full-script""><br/>(GHOST OF A GOOD..."
ncis_season-1/episode-3-Seadog,"<div class=""full-script""><br/>(WHAT I LIKE ABO..."
ncis_season-1/episode-4-The_Immortals,"<div class=""full-script""><br/>(REGGAE MUSIC PL..."
ncis_season-1/episode-5-The_Curse,"<div class=""full-script""><br/>Shoot.<br/><br/>..."


In [51]:
class NLPProcessor:
    
    def __init__(self, vectorizer_class, tokenizer_function, cleaning_function, lemmer_function):
        self.vectorizer = vectorizer_class
        self.tokenizer = tokenizer_function
        self.cleaning_function = cleaning_function
        self.lemmer = lemmer_function
    
    def fit(self, corpus_list_to_fit):
        cleaned_corpus = list(map(self.cleaning_function, corpus_list_to_fit))
#         print(cleaned_corpus)
        tokenized_list = list(map(self.tokenizer, cleaned_corpus))
#         print(tokenized_list)
        lemmed_list = [' '.join(list(map(self.lemmer, item))) for item in tokenized_list]
#         print(lemmed_list)
        return self.vectorizer.fit(lemmed_list)
    
    def transform(self, corpus_list_to_clean):
        cleaned_corpus = list(map(self.cleaning_function, corpus_list_to_clean))
        tokenized_list = list(map(self.tokenizer, cleaned_corpus))
        lemmed_list = [' '.join(list(map(self.lemmer, item))) for item in tokenized_list]
        return pd.DataFrame(self.vectorizer.transform(lemmed_list).toarray(), 
                            columns=self.vectorizer.get_feature_names())

##### NCIS Pipeline

In [70]:
nlp = NLPProcessor(CountVectorizer(stop_words='english'), 
               TreebankWordTokenizer().tokenize, 
               clean_tv_scripts, 
               WordNetLemmatizer().lemmatize)

nlp.fit(ncis_df['dialogue'])
ncis_dtm = nlp.transform(ncis_df['dialogue'])

In [74]:
ncis_dtm_stopmm.info()

<class 'pandas.core.frame.DataFrame'>
Index: 378 entries, ncis_season-1/episode-1-Yankee_White to ncis_season-17/episode-16-Ephemera
Columns: 3523 entries, aa to zoom
dtypes: int64(3523)
memory usage: 10.2+ MB


In [71]:
ncis_big_vocab = nlp.vectorizer
pickle.dump(ncis_big_vocab, open("ncis_cv.pkl", "wb"))

In [80]:
nlp = NLPProcessor(CountVectorizer(stop_words=stop_words,max_df=.95,
                                  min_df=.05), 
               TreebankWordTokenizer().tokenize, 
               clean_tv_scripts, 
               WordNetLemmatizer().lemmatize)

nlp.fit(ncis_df['dialogue'])
ncis_dtm_stopmm = nlp.transform(ncis_df['dialogue'])

In [81]:
ncis_vocab = nlp.vectorizer
pickle.dump(ncis_vocab, open("ncis_cv_stopmm.pkl", "wb"))

In [20]:
ncis_dtm = pd.read_pickle('./pickles/ncis_dtm.pkl')

In [82]:
ncis_dtm_stopmm.index = ncis_df.index

In [26]:
ncis_dtm_stopmm.info()

<class 'pandas.core.frame.DataFrame'>
Index: 378 entries, ncis_season-1/episode-1-Yankee_White to ncis_season-17/episode-16-Ephemera
Columns: 3523 entries, aa to zoom
dtypes: int64(3523)
memory usage: 10.2+ MB


In [21]:
ncis_dtm.info()

<class 'pandas.core.frame.DataFrame'>
Index: 378 entries, ncis_season-1/episode-1-Yankee_White to ncis_season-17/episode-16-Ephemera
Columns: 36208 entries, aa to ﬂoss
dtypes: int64(36208)
memory usage: 104.4+ MB


##### Bosch Pipeline

In [33]:
nlp = NLPProcessor(CountVectorizer(stop_words='english'), 
               TreebankWordTokenizer().tokenize, 
               clean_tv_scripts, 
               WordNetLemmatizer().lemmatize)

nlp.fit(bosch_df['dialogue'])
bosch_dtm = nlp.transform(bosch_df['dialogue'])


[' so here we are in the top of the gonna rain like a bastard tonight youre crazy need it wash all the shit away no sprinkle at most what you think the game gonna get rained out wont go nine thats for sure what are you all of a sudden an amateur meteorologist about time cats shakin and bakin man i think he might have made me hes headed for the metro if i hustle i can get up on him should i sit on his car in case he circles back if he does we got to take him tonight  backup see how it plays out right behind you mariachi plaza carnal no te me esconde alga despacio despacio policia show me your hands hombre manos muestrame tus manos freeze bosch chief in gods name bosch another one what the fuck happened bosch  tis the season sync and correction by solfieri www com this morning testimony in the civil trial of lapd detective harry bosch resumes the stakes are high as the plaintiff seeks unspecified damages for the alleged wrongful death of roberto flores it could be in the millions of doll

[['so', 'here', 'we', 'are', 'in', 'the', 'top', 'of', 'the', 'gon', 'na', 'rain', 'like', 'a', 'bastard', 'tonight', 'youre', 'crazy', 'need', 'it', 'wash', 'all', 'the', 'shit', 'away', 'no', 'sprinkle', 'at', 'most', 'what', 'you', 'think', 'the', 'game', 'gon', 'na', 'get', 'rained', 'out', 'wont', 'go', 'nine', 'thats', 'for', 'sure', 'what', 'are', 'you', 'all', 'of', 'a', 'sudden', 'an', 'amateur', 'meteorologist', 'about', 'time', 'cats', 'shakin', 'and', 'bakin', 'man', 'i', 'think', 'he', 'might', 'have', 'made', 'me', 'hes', 'headed', 'for', 'the', 'metro', 'if', 'i', 'hustle', 'i', 'can', 'get', 'up', 'on', 'him', 'should', 'i', 'sit', 'on', 'his', 'car', 'in', 'case', 'he', 'circles', 'back', 'if', 'he', 'does', 'we', 'got', 'to', 'take', 'him', 'tonight', 'backup', 'see', 'how', 'it', 'plays', 'out', 'right', 'behind', 'you', 'mariachi', 'plaza', 'carnal', 'no', 'te', 'me', 'esconde', 'alga', 'despacio', 'despacio', 'policia', 'show', 'me', 'your', 'hands', 'hombre', 'man

['so here we are in the top of the gon na rain like a bastard tonight youre crazy need it wash all the shit away no sprinkle at most what you think the game gon na get rained out wont go nine thats for sure what are you all of a sudden an amateur meteorologist about time cat shakin and bakin man i think he might have made me he headed for the metro if i hustle i can get up on him should i sit on his car in case he circle back if he doe we got to take him tonight backup see how it play out right behind you mariachi plaza carnal no te me esconde alga despacio despacio policia show me your hand hombre manos muestrame tus manos freeze bosch chief in god name bosch another one what the fuck happened bosch ti the season sync and correction by solfieri www com this morning testimony in the civil trial of lapd detective harry bosch resume the stake are high a the plaintiff seek unspecified damage for the alleged wrongful death of roberto flores it could be in the million of dollar chief cap bo

In [66]:
nlp = NLPProcessor(CountVectorizer(stop_words='english',max_df=.95,min_df=.05), 
               TreebankWordTokenizer().tokenize, 
               clean_tv_scripts, 
               WordNetLemmatizer().lemmatize)

nlp.fit(bosch_df['dialogue'])
bosch_dtm_stopmm = nlp.transform(bosch_df['dialogue'])

In [67]:
bosch_vocab = nlp.vectorizer
pickle.dump(bosch_vocab, open("bosch_cv_stopmm.pkl", "wb"))

##### The Shield Pipeline

In [57]:
nlp = NLPProcessor(CountVectorizer(stop_words='english'), 
               TreebankWordTokenizer().tokenize, 
               clean_tv_scripts, 
               WordNetLemmatizer().lemmatize)

nlp.fit(shield_df['dialogue'])
shield_dtm = nlp.transform(shield_df['dialogue'])

In [76]:
shield_dtm_stopmm.info()

<class 'pandas.core.frame.DataFrame'>
Index: 89 entries, the_shield_season-1/episode-1-Pilot to the_shield_season-7/episode-13-Family_Meeting
Columns: 3472 entries, aah to zone
dtypes: int64(3472)
memory usage: 2.4+ MB


In [59]:
shield_big_vocab = nlp.vectorizer
pickle.dump(shield_big_vocab, open("shield_big_vec_lib.pkl", "wb"))

In [52]:
nlp = NLPProcessor(CountVectorizer(stop_words='english',max_df=.95,min_df=.05), 
               TreebankWordTokenizer().tokenize, 
               clean_tv_scripts, 
               WordNetLemmatizer().lemmatize)

nlp.fit(shield_df['dialogue'])
shield_dtm_stopmm = nlp.transform(shield_df['dialogue'])

In [54]:
shield_vocab = nlp.vectorizer
pickle.dump(shield_vocab, open("shield_cv_stopmm.pkl", "wb"))

In [64]:
bosch_dtm_stopmm.info()

<class 'pandas.core.frame.DataFrame'>
Index: 33 entries, bosch_season-1/episode-1-Chapter_One_Tis_the_Season to bosch_season-5/episode-4-Episode_54
Columns: 9034 entries, aah to zoom
dtypes: int64(9034)
memory usage: 2.3+ MB


In [72]:
ncis_dtm.index = ncis_df.index
bosch_dtm.index = bosch_df.index
shield_dtm.index = shield_df.index
ncis_dtm_stopmm.index = ncis_df.index
bosch_dtm_stopmm.index = bosch_df.index
shield_dtm_stopmm.index = shield_df.index

In [171]:
ncis_dtm.to_pickle('ncis_dtm.pkl')
bosch_dtm.to_pickle('bosch_dtm.pkl')
shield_dtm.to_pickle('shield_dtm.pkl')

##### Pickling Each DTM for Modeling

In [39]:
ncis_dtm.to_pickle('ncis_dtm.pkl')
bosch_dtm.to_pickle('bosch_dtm.pkl')
shield_dtm.to_pickle('shield_dtm.pkl')
ncis_dtm_stopmm.to_pickle('ncis_dtm_stopmm.pkl')
bosch_dtm_stopmm.to_pickle('bosch_dtm_stopmm.pkl')
shield_dtm_stopmm.to_pickle('shield_dtm_stopmm.pkl')

In [83]:
ncis_dtm_stopmm.to_pickle('ncis_dtm_stopmm.pkl')